In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import time
from datetime import datetime
from functools import reduce
import calendar
from scipy.optimize import linear_sum_assignment
from haversine import haversine, Unit
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [17]:
#Define presto credentials
presto_host = 'prime-trino.serving.data.production.internal'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [14]:
#Load the csv with hyderabad area1 hexes
delhi_zone1 = pd.read_csv('/Users/rapido/Desktop/batching_demand/delhi/delhi_location_data_level1_demand_zone1.csv')
delhi_zone1_hexes = list(delhi_zone1['customer_location_hex_8'].unique())
delhi_zone2 = pd.read_csv('/Users/rapido/Desktop/batching_demand/delhi/delhi_location_data_level1_demand_zone2.csv')
delhi_zone2_hexes = list(delhi_zone2['customer_location_hex_8'].unique())
bangalore_zone1 = pd.read_csv('/Users/rapido/Desktop/batching_demand/bangalore_location_data_level1_demand_zone1.csv')
bangalore_zone1_hexes = list(bangalore_zone1['customer_location_hex_8'].unique())
bangalore_zone2 = pd.read_csv('/Users/rapido/Desktop/batching_demand/bangalore_location_data_level1_demand_zone2.csv')
bangalore_zone2_hexes = list(bangalore_zone2['customer_location_hex_8'].unique())
print(delhi_zone1_hexes)
print(delhi_zone2_hexes)
print(bangalore_zone1_hexes)
print(bangalore_zone2_hexes)

['883da1034dfffff', '883da111a1fffff', '883da111a3fffff', '883da111a7fffff', '883da111a9fffff', '883da111adfffff', '883da11a83fffff', '883da11a87fffff', '883da11a91fffff', '883da11a99fffff', '883da11a9bfffff', '883da11abdfffff', '883da11ad5fffff', '883da10341fffff', '883da10345fffff', '883da10349fffff', '883da11185fffff', '883da111a5fffff', '883da111abfffff', '883da111b5fffff', '883da111bdfffff', '883da111e7fffff', '883da11a81fffff', '883da11a85fffff', '883da11a89fffff', '883da11a8bfffff', '883da11a93fffff', '883da11a95fffff', '883da11a97fffff', '883da11a9dfffff', '883da11aa3fffff', '883da11aabfffff', '883da11ab1fffff', '883da11ab5fffff', '883da11ab9fffff', '883da11ac3fffff', '883da11ad1fffff', '883da11ad3fffff', '883da11ad7fffff', '883da11adbfffff', '883da11addfffff']
['883da106c1fffff', '883da106c9fffff', '883da1a939fffff', '883da1a93dfffff', '883da106c3fffff', '883da106c5fffff', '883da106c7fffff', '883da106cbfffff', '883da106cdfffff', '883da1a903fffff', '883da1a907fffff', '883da1a91

In [18]:
#Query data from order_logs_snapshot

q = """
select order_id, city_name, customer_location_hex_8, hour, service_obj_service_name, yyyymmdd,
order_status, event_type, spd_fraud_flag
from orders.order_logs_immutable
where yyyymmdd >= '20220919'
and yyyymmdd <= '20221016'
and service_obj_service_name = 'Link'
and event_type in ('accepted', 'rider_busy', 'rider_reject')
and customer_location_hex_8 in ('883da1034dfffff', '883da111a1fffff', '883da111a3fffff', '883da111a7fffff', '883da111a9fffff', '883da111adfffff',
 '883da11a83fffff', '883da11a87fffff', '883da11a91fffff', '883da11a99fffff', '883da11a9bfffff', '883da11abdfffff', '883da11ad5fffff', '883da10341fffff',
  '883da10345fffff', '883da10349fffff', '883da11185fffff', '883da111a5fffff', '883da111abfffff', '883da111b5fffff', '883da111bdfffff', '883da111e7fffff',
   '883da11a81fffff', '883da11a85fffff', '883da11a89fffff', '883da11a8bfffff', '883da11a93fffff', '883da11a95fffff', '883da11a97fffff', '883da11a9dfffff',
    '883da11aa3fffff', '883da11aabfffff', '883da11ab1fffff', '883da11ab5fffff', '883da11ab9fffff', '883da11ac3fffff', '883da11ad1fffff', '883da11ad3fffff',
     '883da11ad7fffff', '883da11adbfffff', '883da11addfffff', '883da106c1fffff', '883da106c9fffff', '883da1a939fffff', '883da1a93dfffff', '883da106c3fffff',
      '883da106c5fffff', '883da106c7fffff', '883da106cbfffff', '883da106cdfffff', '883da1a903fffff', '883da1a907fffff', '883da1a915fffff', '883da1a923fffff',
       '883da1a927fffff', '883da1a92bfffff', '883da1a931fffff', '883da1a935fffff', '883da1a93bfffff', '8861892087fffff', '8861892097fffff', '88618920a3fffff',
        '88618920b1fffff', '88618920b9fffff', '88618920bbfffff', '88618920bdfffff', '8861892091fffff', '8861892093fffff', '8861892095fffff', '88618920a7fffff',
         '88618920abfffff', '88618920b3fffff', '88618920b5fffff', '88618920b7fffff', '8861892e49fffff', '8861892437fffff', '8861892589fffff', '88618925c3fffff',
          '88618925c7fffff', '88618925cbfffff', '88618925e9fffff', '8861892431fffff', '8861892433fffff', '8861892435fffff', '8861892513fffff', '8861892517fffff',
           '8861892581fffff', '886189258bfffff', '886189258dfffff', '88618925c1fffff', '88618925c5fffff', '88618925c9fffff', '88618925d5fffff', '88618925d9fffff',
            '88618925ddfffff', '88618925e1fffff', '88618925ebfffff', '88618925edfffff')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

DatabaseError: {'message': 'Query exceeded the maximum execution time limit of 20.00m', 'errorCode': 131075, 'errorName': 'EXCEEDED_TIME_LIMIT', 'errorType': 'INSUFFICIENT_RESOURCES', 'failureInfo': {'type': 'io.trino.spi.TrinoException', 'message': 'Query exceeded the maximum execution time limit of 20.00m', 'suppressed': [], 'stack': ['io.trino.execution.QueryTracker.enforceTimeLimits(QueryTracker.java:187)', 'io.trino.execution.QueryTracker.lambda$start$0(QueryTracker.java:89)', 'com.google.common.util.concurrent.MoreExecutors$ScheduledListeningDecorator$NeverSuccessfulListenableFutureTask.run(MoreExecutors.java:730)', 'java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)', 'java.base/java.util.concurrent.FutureTask.runAndReset(FutureTask.java:305)', 'java.base/java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask.run(ScheduledThreadPoolExecutor.java:305)', 'java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.jav

In [ ]:
#Load a copy of the df
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag
0,6362f39d8143791dd512cae5,0.0582,8860a259b9fffff,17.441437,78.391579,Hyderabad,8860a259b9fffff,8860a259a1fffff,17.455869,78.403663,17.441444,78.391541,041757,04,0415,Link,04:00,4,20221103,arrived,Vehicle Issue,1667429378024,arrived,61788ce2fbce66032da33e95,None
1,6362f95152c77607a229956e,NaN,,NaN,NaN,Hyderabad,8860a25991fffff,8860a259b7fffff,17.455235,78.376937,17.431190,78.375725,044217,04,0430,Link,04:59,4,20221103,new,None,1667430737511,order_create,,None
2,6362f79db8b83a62a700db61,0.6720,8860a24a69fffff,17.437738,78.365540,Hyderabad,8860a24a65fffff,8860a24a69fffff,17.437738,78.365540,17.447717,78.363679,043501,04,0430,Link,04:59,4,20221103,dropped,None,1667430704589,order_updated,5c31eddd4a267149c767770d,False
3,6362f8cb8143791dd512ccc7,0.1200,8860a259e9fffff,17.439030,78.425659,Hyderabad,8860a25983fffff,8860a259e9fffff,17.439030,78.425659,17.441048,78.392532,044003,04,0430,Link,04:59,4,20221103,dropped,None,1667432077781,rider_rated,63313e4fe90ec6d73b77a508,False
4,6362f1f552e5fb3bb7c5531b,0.9240,8860a259a1fffff,17.458858,78.404312,Hyderabad,8860a259b1fffff,8860a259a1fffff,17.458858,78.404312,17.452029,78.387510,041053,04,0400,Link,04:00,4,20221103,dropped,Unsafe Pickup area,1667455851610,customer_rated,5e61c0298ec627e7ad977331,False


In [ ]:
#Create a count column
df['count'] = 1

#Create a column to identify the zone
df['zone_name'] = np.where(df['customer_location_hex_8'].isin(delhi_zone1_hexes), 'delhi_zone1',
 np.where(df['customer_location_hex_8'].isin(delhi_zone2_hexes), 'delhi_zone2',
  np.where(df['customer_location_hex_8'].isin(bangalore_zone1_hexes), 'bangalore_zone1', 'bangalore_zone2')))

#Create a column to identify total pings and accepted ping
df['total_pings'] = np.where(df['event_type'].isin(['accepted', 'rider_busy', 'rider_reject']), 1, 0)
df['accepted_pings'] = np.where(df['event_type'].isin(['accepted']), 1, 0)
df['rejected_pings'] = np.where(df['event_type'].isin(['rider_reject']), 1, 0)
df['busy_pings'] = np.where(df['event_type'].isin(['rider_busy']), 1, 0)
df_me = df[(df['order_status'] == 'dropped') & (df['spd_fraud_flag'] != True)]

#Groupby on date and hour to get the APR and ME on an hourly basis
apr_hyderabad = df.groupby(['zone_name', 'yyyymmdd', 'hour'])['total_pings', 'accepted_pings', 'rejected_pings', 'busy_pings'].sum().reset_index()
apr_hyderabad['APR'] = round(apr_hyderabad['accepted_pings']/apr_hyderabad['total_pings'], 4)
apr_hyderabad['rejected_pings%'] = round(apr_hyderabad['rejected_pings']/apr_hyderabad['total_pings'], 4)
apr_hyderabad['busy_pings%'] = round(apr_hyderabad['busy_pings']/apr_hyderabad['total_pings'], 4)
me_hyderabad = df_me.groupby(['zone_name', 'yyyymmdd', 'hour'])['order_id'].nunique().reset_index()
me_hyderabad = me_hyderabad.rename(columns={'order_id':'net_orders'})
apr_hyderabad = apr_hyderabad.merge(me_hyderabad, on = ['zone_name', 'yyyymmdd', 'hour'], how = 'left')
apr_hyderabad['ME'] = round(apr_hyderabad['total_pings']/apr_hyderabad['net_orders'], 2)
apr_hyderabad.head()

,zone_name,yyyymmdd,hour,total_pings,accepted_pings,APR,net_orders,ME
0,zone1,20221103,00,362,247,0.6823,140,2.59
1,zone1,20221103,01,219,146,0.6667,63,3.48
2,zone1,20221103,02,203,132,0.6502,54,3.76
3,zone1,20221103,03,231,133,0.5758,40,5.78
4,zone1,20221103,04,183,96,0.5246,38,4.82
5,zone1,20221103,05,247,147,0.5951,81,3.05
6,zone1,20221103,06,388,313,0.8067,250,1.55
7,zone1,20221103,07,667,569,0.8531,472,1.41
8,zone1,20221103,08,2710,1790,0.6605,1214,2.23
9,zone1,20221103,09,4650,2842,0.6112,1708,2.72


In [ ]:
apr_hyderabad.to_csv('apr_delhi_zone1.csv', index = False)

In [10]:
#Query data from driving_distance_and_time
q_dist = """
select * from experiments.driving_distance_and_time
where city_name in ('Hyderabad')
and hex_resolution = 8
"""

#Load the data into pandas table
df_distance_copy = pd.read_sql(q_dist, connection)

In [11]:
#View the distance dataset
df_distance = df_distance_copy.copy()
df_distance.head()

,day_name,destination_hex,distance_in_km,hex_resolution,num_observations,ridetime_in_minutes,run_date,source_hex,time_period,city_name
0,Saturday,8860a24a1dfffff,22.220,8,1,35.100000,20221025,8860a24021fffff,morning_peak,Hyderabad
1,Saturday,8860a251cdfffff,24.012,8,1,44.866667,20221025,8860a24021fffff,rest_morning,Hyderabad
2,Tuesday,8860a258a9fffff,17.614,8,1,37.650000,20221025,8860a24021fffff,evening_peak,Hyderabad
3,Wednesday,8860a241adfffff,15.508,8,1,38.666667,20221025,8860a24027fffff,evening_peak,Hyderabad
4,Saturday,8860a24065fffff,2.160,8,2,2.363333,20221025,8860a24029fffff,evening_peak,Hyderabad


In [12]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['weekday_name'] = df['weekday'].apply(lambda x: calendar.day_name[(x-1)])

#Create a column to get the time of the day
df['time_period'] = np.where(df['hour'].isin(['08', '09', '10', '11']), 'morning_peak', np.where(df['hour'].isin(['17', '18', '19', '20', '21']), 'evening_peak',
    np.where(df['hour'].isin(['12, 13, 14, 15, 16']), 'afternoon', np.where(df['hour'].isin(['00', '01', '02', '03', '04', '05', '06', '07']),
     'rest_morning', 'rest_evening'))))


#Make a column to combine cust_lat_long and cap_lat_long
df['cust_lat_long'] = df[['customer_location_latitude', 'customer_location_longitude']].values.tolist()
df['cap_lat_long'] = df[['captain_location_latitude', 'captain_location_longitude']].values.tolist()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,count,zone_name,total_pings,accepted_pings,second,minute,date,weekday_name,time_period,cust_lat_long,cap_lat_long
0,632c3806afd7ae59864b6887,2.5820,8860a24a61fffff,17.444494,78.363281,Hyderabad,8860a24a6dfffff,8860a24a61fffff,17.444494,78.363281,17.446301,78.371201,155510,15,1545,Link,15:59,4,20220922,dropped,None,1663842949878,order_updated,5c6f69a535478b560278d490,False,1,zone1,0,0,10,55,22,Thursday,rest_evening,"[17.446300506591797, 78.37120056152344]","[17.444494247436523, 78.36328125]"
1,632c2c875b830c0dfa2ca9ed,0.7800,8860a259b9fffff,17.444342,78.385852,Hyderabad,8860a259b9fffff,8860a25995fffff,17.436895,78.386308,17.443260,78.386841,150607,15,1500,Link,15:00,4,20220922,onTheWay,Wrong Pickup Location,1663839416052,accepted,63133c414e349c682907918f,None,1,zone1,1,1,07,06,22,Thursday,rest_evening,"[17.443260192871094, 78.3868408203125]","[17.4443417, 78.3858517]"
2,632c2cf7853c020ac79bbd49,0.6020,8860a2582dfffff,17.407991,78.444099,Hyderabad,8860a25953fffff,8860a2582dfffff,17.407991,78.444099,17.413277,78.447975,150759,15,1500,Link,15:00,4,20220922,dropped,None,1663840635891,rider_rated,60ddde9a569b2654c76e9b15,False,1,zone2,0,0,59,07,22,Thursday,rest_evening,"[17.413277, 78.447975]","[17.407991409301758, 78.44409942626953]"
3,632c2def853c020ac79bbfcc,NaN,,NaN,NaN,Hyderabad,8860a25997fffff,8860a24b5bfffff,17.457595,78.363949,17.436134,78.376465,151207,15,1500,Link,15:00,4,20220922,requested,None,1663839727965,order_requested,None,None,1,zone1,0,0,07,12,22,Thursday,rest_evening,"[17.436134338378906, 78.37646484375]","[nan, nan]"
4,632c2e3a87031a328ca78ded,1.3542,8860a24b29fffff,17.501276,78.378052,Hyderabad,8860a259b1fffff,8860a259b1fffff,17.451040,78.386229,17.448597,78.385696,151322,15,1500,Link,15:00,4,20220922,arrived,None,1663840130649,arrived,62c11f9dd9dd8974f0711bae,None,1,zone1,0,0,22,13,22,Thursday,rest_evening,"[17.448596954345703, 78.38569641113281]","[17.50127601623535, 78.3780517578125]"


In [13]:
#Create 20-sec batches
df['second'] = df['second'].astype(float)
df['minute'] = df['minute'].astype(float)
df['20_sec_batch'] = pd.cut(df['second'], bins = [-1, 20, 40, 61], labels = ['0-20', '20-40', '40-60'])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,count,zone_name,total_pings,accepted_pings,second,minute,date,weekday_name,time_period,cust_lat_long,cap_lat_long,20_sec_batch
0,632c3806afd7ae59864b6887,2.5820,8860a24a61fffff,17.444494,78.363281,Hyderabad,8860a24a6dfffff,8860a24a61fffff,17.444494,78.363281,17.446301,78.371201,155510,15,1545,Link,15:59,4,20220922,dropped,None,1663842949878,order_updated,5c6f69a535478b560278d490,False,1,zone1,0,0,10.0,55.0,22,Thursday,rest_evening,"[17.446300506591797, 78.37120056152344]","[17.444494247436523, 78.36328125]",0-20
1,632c2c875b830c0dfa2ca9ed,0.7800,8860a259b9fffff,17.444342,78.385852,Hyderabad,8860a259b9fffff,8860a25995fffff,17.436895,78.386308,17.443260,78.386841,150607,15,1500,Link,15:00,4,20220922,onTheWay,Wrong Pickup Location,1663839416052,accepted,63133c414e349c682907918f,None,1,zone1,1,1,7.0,6.0,22,Thursday,rest_evening,"[17.443260192871094, 78.3868408203125]","[17.4443417, 78.3858517]",0-20
2,632c2cf7853c020ac79bbd49,0.6020,8860a2582dfffff,17.407991,78.444099,Hyderabad,8860a25953fffff,8860a2582dfffff,17.407991,78.444099,17.413277,78.447975,150759,15,1500,Link,15:00,4,20220922,dropped,None,1663840635891,rider_rated,60ddde9a569b2654c76e9b15,False,1,zone2,0,0,59.0,7.0,22,Thursday,rest_evening,"[17.413277, 78.447975]","[17.407991409301758, 78.44409942626953]",40-60
3,632c2def853c020ac79bbfcc,NaN,,NaN,NaN,Hyderabad,8860a25997fffff,8860a24b5bfffff,17.457595,78.363949,17.436134,78.376465,151207,15,1500,Link,15:00,4,20220922,requested,None,1663839727965,order_requested,None,None,1,zone1,0,0,7.0,12.0,22,Thursday,rest_evening,"[17.436134338378906, 78.37646484375]","[nan, nan]",0-20
4,632c2e3a87031a328ca78ded,1.3542,8860a24b29fffff,17.501276,78.378052,Hyderabad,8860a259b1fffff,8860a259b1fffff,17.451040,78.386229,17.448597,78.385696,151322,15,1500,Link,15:00,4,20220922,arrived,None,1663840130649,arrived,62c11f9dd9dd8974f0711bae,None,1,zone1,0,0,22.0,13.0,22,Thursday,rest_evening,"[17.448596954345703, 78.38569641113281]","[17.50127601623535, 78.3780517578125]",20-40


In [14]:
#Filter the df on the pings only
df_filter = df[(df['event_type'] == 'rider_busy') | (df['event_type'] == 'rider_reject') | (df['event_type'] == 'accepted')]

In [17]:
#Create a column to combine customer and captain hex
df_filter['source_destination'] = df_filter['customer_location_hex_8'] + df_filter['captain_location_hex_8']
df_distance['source_destination'] = df_distance['source_hex'] + df_distance['destination_hex']
distance_dict = dict(zip(df_distance['source_destination'], df_distance['distance_in_km']))

#Get the distance between customer and each captain available
customer_hexes = df_filter['source_destination'].values.tolist()
cap_lat_long = df_filter['cap_lat_long'].values.tolist()
cust_lat_long = df_filter['cust_lat_long'].values.tolist()

In [21]:
captain_distances = []
for hex in range(0, len(customer_hexes)):
    key_hex = customer_hexes[hex]
    if key_hex in distance_dict:
        individual_distance = distance_dict[key_hex]
    else:
        individual_distance = round(haversine(cap_lat_long[hex], cust_lat_long[hex]), 3)
    captain_distances.append(individual_distance)

In [22]:
#Add the distances of the captains
df_filter['captain_distances'] = captain_distances
df_filter['captain_rank'] = df_filter.groupby(['order_id'])['captain_distances'].rank(ascending=True)
df_filter.head()


,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,count,zone_name,total_pings,accepted_pings,second,minute,date,weekday_name,time_period,cust_lat_long,cap_lat_long,20_sec_batch,source_destination,captain_distances,captain_rank
1,632c2c875b830c0dfa2ca9ed,0.78,8860a259b9fffff,17.444342,78.385852,Hyderabad,8860a259b9fffff,8860a25995fffff,17.436895,78.386308,17.443260,78.386841,150607,15,1500,Link,15:00,4,20220922,onTheWay,Wrong Pickup Location,1663839416052,accepted,63133c414e349c682907918f,None,1,zone1,1,1,7.0,6.0,22,Thursday,rest_evening,"[17.443260192871094, 78.3868408203125]","[17.4443417, 78.3858517]",0-20,8860a259b9fffff8860a259b9fffff,1.481075,1.5
28,632c3871d2293823b6aa044e,NaN,8860a24a67fffff,17.449088,78.357086,Hyderabad,8860a24a67fffff,8860a259bbfffff,17.440993,78.376747,17.447178,78.359938,155657,15,1545,Link,15:59,4,20220922,new,None,1663842448700,rider_busy,5bfe5919cb3d8419414747d6,None,1,zone1,1,0,57.0,56.0,22,Thursday,rest_evening,"[17.447178173846197, 78.35993842858647]","[17.44908763100006, 78.3570858836174]",40-60,8860a24a67fffff8860a24a67fffff,0.412000,1.0
33,632c2ef987031a328ca78fca,NaN,8860a25907fffff,17.446394,78.458916,Hyderabad,8860a25907fffff,8860a25b17fffff,17.386137,78.463865,17.446400,78.458916,151633,15,1515,Link,15:00,4,20220922,onTheWay,None,1663840023815,accepted,6307a6e372f91f6e45012b11,None,1,zone2,1,1,33.0,16.0,22,Thursday,rest_evening,"[17.446399688720703, 78.45891571044922]","[17.446393786822533, 78.45891561359167]",20-40,8860a25907fffff8860a25907fffff,1.461563,1.0
35,632c31bdafd7ae59864b58b9,NaN,8860a259abfffff,17.451311,78.397175,Hyderabad,8860a259abfffff,8860a259b1fffff,17.452822,78.380354,17.453842,78.399216,152821,15,1515,Link,15:00,4,20220922,onTheWay,None,1663840706075,accepted,631775b8304c3053a2a97719,None,1,zone1,1,1,21.0,28.0,22,Thursday,rest_evening,"[17.453842163085938, 78.39921569824219]","[17.4513107, 78.3971746]",20-40,8860a259abfffff8860a259abfffff,2.906000,1.0
40,632c3621aa9dd231fe21073e,NaN,8860a259b7fffff,17.456727,78.377256,Hyderabad,8860a259b7fffff,8860a25999fffff,17.422177,78.379691,17.456425,78.376572,154705,15,1545,Link,15:59,4,20220922,new,None,1663841861955,rider_busy,5d53ea48bde10e0d543dfcfe,None,1,zone1,1,0,5.0,47.0,22,Thursday,rest_evening,"[17.456424713134766, 78.37657165527344]","[17.456727280574306, 78.3772560954094]",0-20,8860a259b7fffff8860a259b7fffff,2.927000,1.5


In [25]:
#Filter the df on captains ranked 2 and above
df_filter = df_filter[df_filter['captain_rank'] >= 2]
df_filter_zone1 = df_filter[df_filter['zone_name'] == 'zone1']
df_filter_zone2 = df_filter[df_filter['zone_name'] == 'zone2']

In [26]:
#Filter the df by hour to get hourly reduction in fm
hours = {}
dates = {}
for date in df_filter_zone1['date'].unique().tolist():
    df_date = df_filter_zone1[df_filter_zone1['date'] == date]
    for hour in df_date['hour'].unique().tolist():
        df_hourly = df_date[df_date['hour'] == hour]
        df_distance_day = df_distance[df_distance['day_name'] == df_hourly['weekday_name'].values[0]]

        #Get the captains available for every 20_sec_batch
        captains_available = df_hourly.groupby(['hour', 'minute', '20_sec_batch'])['captain_location_hex_8'].apply(list).reset_index()
        captains_available2 = df_hourly.groupby(['hour', 'minute', '20_sec_batch'])['cap_lat_long'].apply(list).reset_index()
        captains_available = captains_available.rename(columns = {'captain_location_hex_8':'cap_location_hexes'})
        captains_available2 = captains_available2.rename(columns = {'cap_lat_long':'cap_location_lat_long'})
        #captains_available['number_of_captains'] = captains_available['cap_location_hexes'].apply(lambda x: len(x))
        captains_available = captains_available.merge(captains_available2, on = ['hour', 'minute', '20_sec_batch'], how = 'left')

        #Merge the captain locations with our df
        df_hourly = df_hourly.merge(captains_available, on = ['hour', 'minute', '20_sec_batch'], how = 'left')
        
        #Filter the distance df based on the hour
        if hour in ['08', '09', '10', '11']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'morning_peak']
        elif hour in ['17', '18', '19', '20', '21']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'evening_peak']
        elif hour in ['12, 13, 14, 15, 16']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'afternoon']
        elif hour in ['00', '01', '02', '03', '04', '05', '06', '07']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'rest_morning']
        else:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'rest_evening']

        #Combine the source and destination hex and make a dictionary to get distance between each hex combo
        df_distance_filter['source_destination'] = df_distance_filter['source_hex'] + df_distance_filter['destination_hex']
        distance_dict = dict(zip(df_distance_filter['source_destination'], df_distance_filter['distance_in_km']))

        #Get the distance between customer and each captain available
        customer_hexes = df_hourly['customer_location_hex_8'].values.tolist()
        captain_hexes = df_hourly['cap_location_hexes'].values.tolist()
        cap_lat_long = df_hourly['cap_location_lat_long'].values.tolist()
        cust_lat_long = df_hourly['cust_lat_long'].values.tolist()

        i = 0
        captain_distances = []
        for x in captain_hexes:
            customer_distance = []
            for hex in range(0, len(x)):
                key_hex = x[hex] + customer_hexes[i]
                if key_hex in distance_dict:
                    individual_distance = distance_dict[key_hex]
                else:
                    individual_distance = round(haversine(cap_lat_long[i][hex], cust_lat_long[i]), 3)
                customer_distance.append(individual_distance)
            captain_distances.append(customer_distance)
            i += 1

        #Add columns for captain distances and etas to each customer
        df_hourly['captain_distances'] = captain_distances
        df_hourly['captain_rank'] = df_hourly.groupby(['order_id'])['captain_distances'].rank(ascending = True)

        #Filter out the first two ranks for each order_id

        #Get the nxn matrix for each 20_sec_batch
        df_batches = df_hourly.groupby(['minute', '20_sec_batch'])['captain_distances'].apply(list).reset_index()
        df_batches = df_batches.dropna()

        #Get the row_index and column index suitable for each customer
        optimal_distance = []
        for x in df_batches['captain_distances'].values.tolist():
            try:
                row_ind, col_ind = linear_sum_assignment(x)
            except Exception as e:
                continue
            optimal_distance.append(col_ind)

        #Get the optimal distance for each customer
        captain_customer_dist = df_batches['captain_distances'].values.tolist()
        i = 0
        min_distances = []
        for x in captain_customer_dist:
            j = 0
            for y in x:
                if type(y) == float:
                    continue
                min_distance = y[optimal_distance[i][j]]
                j += 1
                min_distances.append(min_distance)
            i += 1

        #Add the new_dist to the df
        df_hourly['new_FM'] = min_distances

        # #Calculate old_FM
        # df_hourly['cap_cust_hex'] = df_hourly['customer_location_hex_8'] + df_hourly['captain_location_hex_8']
        # captain_customer_dist = []
        # for x in df_hourly['cap_cust_hex'].values.tolist():
        #     if x in distance_dict:
        #         distance = distance_dict[x]
        #     else:
        #         distance = 'NA'
        #     captain_customer_dist.append(distance)

        # cap_lat_long = df_hourly['cap_lat_long'].values.tolist()
        # cust_lat_long = df_hourly['cust_lat_long'].values.tolist()

        # haversine_cust_cap = []
        # for i in range(0, len(cap_lat_long)):
        #     haversine_distance = round(haversine(cap_lat_long[i], cust_lat_long[i]), 3)
        #     haversine_cust_cap.append(haversine_distance)

        # for i in range(0, len(captain_customer_dist)):
        #     if captain_customer_dist[i] == 'NA':
        #         captain_customer_dist[i] = haversine_cust_cap[i]

        # df_hourly['old_FM'] = captain_customer_dist

        #Get the mean FM for each 20_sec_batch (old and new)
        fm_min = df_hourly.groupby(['minute', '20_sec_batch'])['new_FM'].mean().reset_index()

        #Add a column for percentage increase/decrease
        #fm_min['%_change_dist'] = (fm_min['new_FM']/fm_min['old_FM'] - 1)

        #Get the percentage change in FM
        #fm_change = round((fm_min['new_FM'].mean()/fm_min['old_FM'].mean() - 1), 4)
        hours[hour] = fm_min
    dates[date] = hours

KeyboardInterrupt: 

In [ ]:
#Create a df from the dates dict
hourly_decrease = pd.DataFrame(dates).T
hourly_decrease = hourly_decrease.reset_index()
hourly_decrease = pd.melt(hourly_decrease, id_vars='index')
hourly_decrease.columns = ['date', 'hour', 'New_FM']
hourly_decrease = hourly_decrease.sort_values(by = ['date', 'hour'], ascending=True)
hourly_decrease